# Внимание.
Перед загрузкой моделей необходимо загрузить модели и исходные данные в среду выполнения Google Colab (раздел "Файлы" на левой панели инструментов).
Исходные данные должны быть в файле 'dataset.xlsx'.
# Для выполнения ячеек нажмите на верхней панели инструментов "Среда выполнения" -> "Выполнить все" или нажмите на Ctrl+F9.

# Внимание.
Итоговый файл с прогнозами моделей необходимо скачать на Ваше устройство (файл 'dataset_results.xlsx' раздела "Файлы" на левой панели инструментов).
В него добавлены пять столбцов (четыре с прогнозами моделей и один со средним значением прогнозов)

In [1]:
#@title Загрузка библиотек

!pip install catboost

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from joblib import load

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier

In [3]:
#@title Загрузка моделей и исходных данных

model_forest = load('logreg.pkl')
model_logreg = load('forest.pkl')
model_svc = load('svc.pkl')
model_catboost = CatBoostClassifier()
model_catboost.load_model("catboost")
df = pd.read_excel('dataset.xlsx')

In [4]:
# копия датафрейма для вывода вероятностей
df_spr = df.copy()

In [5]:
forest_predict = model_forest.predict(df)
logreg_predict = model_logreg.predict(df)
svc_predict = model_svc.predict(df)
catboost_predict = model_catboost.predict(df)

In [6]:
df['operation_forest_predict'] = forest_predict
df['operation_logreg_predict'] = logreg_predict
df['operation_svc_predict'] = svc_predict
df['operation_catboost_predict'] = catboost_predict

# добавление столбца с мажоритарным голосованием моделей
df['majority_vote'] = df[['operation_forest_predict', 'operation_logreg_predict', 
                          'operation_svc_predict', 'operation_catboost_predict']].mean(axis=1)

In [7]:
df.to_excel('dataset_results.xlsx')

In [8]:
#@title Справочно. Ниже приведены вероятности прогнозов моделей (модель svc не поддерживает функцию)

probs = pd.DataFrame()
prob_forest = pd.DataFrame(model_forest.predict_proba(df_spr))
probs['prob_forest'] = prob_forest.max(axis=1)
prob_logreg = pd.DataFrame(model_logreg.predict_proba(df_spr))
probs['prob_logreg'] = prob_logreg.max(axis=1)
prob_catboost = pd.DataFrame(model_catboost.predict_proba(df_spr))
probs['prob_catboost'] = prob_catboost.max(axis=1)
probs

,prob_forest,prob_logreg,prob_catboost
0,0.793224,0.98,0.806600
1,0.930230,1.00,0.979723
2,0.899112,1.00,0.978171
3,0.697685,1.00,0.951196
4,0.804841,1.00,0.944264
5,0.908197,1.00,0.985959
6,0.728252,0.93,0.885124
7,0.815241,1.00,0.884904
8,0.558860,0.91,0.795901
